# 1. Points of Interest

A Point of Interest (POI) is a feature occupying a particular point on a map.
They can be used to indicate anything a map reader might be interested in and want to locate.

POI data is plentiful, but comprehensive open data is hard to come by.
[OpenStreetMap (OSM)](https://www.openstreetmap.org/) is the most often recommended open database; however, its POI coverage and quality depend entirely on community contributions.
Because of this, things can be inconsistently or insuﬀiciently tagged,
giving an area the false impression of there being fewer features than there really are.

In this notebook, I compile my own POI list by using the city’s oﬀicial platform for open data, [_Données Ouvertes de Montréal (DOM)_ (Montreal Open Data)](https://donnees.montreal.ca/) together with OSM data.
POIs relevant for the 15-minute city are chosen and assigned to these thematic groups:

1. Food and drink
2. Groceries
3. Healthcare
4. Education
5. Recreation
6. Commerce
7. Community
8. Culture
9. Mobility
10. Civic spaces
11. Government services

In [ ]:
from final_project.config import CRS, FEATURES_DIR, META_DIR, RAW_DIR
from final_project.data import poi

## _Données Ouvertes de Montréal_

DOM provides two relevant data sets:

- [Lieux d’intérêt (Places of interest)](https://donnees.montreal.ca/dataset/lieux-d-interet): Cultural, commercial, recreational, and public service places of interest classified by the _Montréal à Pied_ (Montreal on Foot) project.
- [_Établissements alimentaires_ (Food establishments)](https://donnees.montreal.ca/dataset/etablissements-alimentaires): Establishments that sell food and that are under the responsibility of the _Division de l’inspection des aliments de la Ville de Montréal_ (Food Inspection Division of the City of Montreal).

The first data set covers only the administrative city of Montreal; the second covers all municipalities on the island.

Ensure these CSVs are extracted to the data directory with the required names.
There is also JSON metadata necessary to map French text to their English translations.

In [ ]:
places_file = RAW_DIR / 'lieux_d_interet.csv'
establishments_file = RAW_DIR / 'etablissements_alimentaires.csv'
translation_metadata = META_DIR / 'translations.json'

Read both files and select the POIs that fit the thematic groups above.

In [ ]:
dom_pois = poi.load_dom_pois(
    places_file,
    establishments_file,
    translation_metadata)
dom_pois.head()

## OpenSteetMap

I combine the on-island suburbs into a single area and query OSM for features in this area.
OSM POIs can be tagged by key-value pairs where the key is a top-level tag and the value is a specific type (for example, `amenity=cafe`).
I choose tags and types to most closely match what POIs were pulled from the _Lieux d’intérêt_ data set but were not already pulled from the _Établissements alimentaires_ data set.
Since this process queries OSM individually for each municipality, if uncached, it can take up to one minute.

In [ ]:
osm_pois = poi.load_osm_pois()
osm_pois.head()

## Combining POIs

Combine the two data frames into a single set of POIs.
There is additional JSON metadata that maps each POI type, the naming of which differs between DOM and OSM, to its thematic group.

In [ ]:
mapping_metadata = META_DIR / 'group_mappings.json'

all_pois = poi.combine_pois(dom_pois, osm_pois, mapping_metadata)
all_pois = all_pois.to_crs(CRS)
all_pois.head()

Save this data for feature engineering in the next steps.

In [ ]:
all_pois.to_file(FEATURES_DIR / 'pois.geojson', driver='GeoJSON')